# Transformação dos dados das instalações

In [0]:
### Importes necessários

from pyspark.sql.functions import split, col


In [0]:
# Caminho do arquivo Delta Lake
file_path = "/mnt/dados/camada_bronze_instalacoes"

# Ler o arquivo Delta Lake
df = spark.read.format("delta").load(file_path)

# Visualizar as primeiras 5 linhas do DataFrame
df.show()
df.printSchema()

+------+--------------------+--------------------+---------+
|codigo|          instalacao|         coordenadas| operador|
+------+--------------------+--------------------+---------+
|   COV|    Terminal Coveñas|9.409136280745889...|   Ocensa|
|   LAG|Estacion La Granjita|8.449501151686118...|   Ocensa|
|   CAU|     Planta Caucasia|7.976933967792115...|      ODC|
|   CHI|  Estacion Chiquillo|6.849724465710438...|   Ocensa|
|   VAS|   Estación Vasconia|6.066431800297201...|    CENIT|
|   LAB| ESTACIÓN LA BELLEZA|5.840039734738277...|   Ocensa|
|   MIR| Estación Miraflores|5.187013677738204...|   Ocensa|
|   PAE|       Estacion Páez|5.071094644914588...|   Ocensa|
|   ELP|Estación El Porvenir|4.921052391552144...|   Ocensa|
|   CUS|    Estacion Cusiana|5.002134841542299...|   Ocensa|
|   CUP|        CPF Cupiagua|5.2090068492098, ...|Ecopetrol|
+------+--------------------+--------------------+---------+

root
 |-- codigo: string (nullable = true)
 |-- instalacao: string (nullable = true)

Vemos que as coordenadas (latitude, longitude) estão em apenas 1 coluna do tipo string. Vamos separar em 2 colunas.

In [0]:

# Divide a coluna 'Coordenadas' em duas colunas separadas
df = df.withColumn("latitude_instalacao", split(col("coordenadas"), ",").getItem(0).cast("double"))
df = df.withColumn("longitude_instalacao", split(col("coordenadas"), ",").getItem(1).cast("double"))

# Remove a coluna original 'Coordenadas'
df = df.drop("Coordenadas")

# Mostra o resultado
df.show()
df.printSchema()

+------+--------------------+---------+-------------------+--------------------+
|codigo|          instalacao| operador|latitude_instalacao|longitude_instalacao|
+------+--------------------+---------+-------------------+--------------------+
|   COV|    Terminal Coveñas|   Ocensa|  9.409136280745889|  -75.70015297027714|
|   LAG|Estacion La Granjita|   Ocensa|  8.449501151686118|  -75.50994443266501|
|   CAU|     Planta Caucasia|      ODC|  7.976933967792115|  -75.22089861123531|
|   CHI|  Estacion Chiquillo|   Ocensa|  6.849724465710438|  -74.48568700509468|
|   VAS|   Estación Vasconia|    CENIT|  6.066431800297201|  -74.55827667262115|
|   LAB| ESTACIÓN LA BELLEZA|   Ocensa|  5.840039734738277|   -73.9661266490697|
|   MIR| Estación Miraflores|   Ocensa| 5.1870136777382045|  -73.16287230331245|
|   PAE|       Estacion Páez|   Ocensa|  5.071094644914588|  -73.06434451310464|
|   ELP|Estación El Porvenir|   Ocensa|  4.921052391552144|   -72.9326776922106|
|   CUS|    Estacion Cusiana

Certo. Agora podemos persistir nosso novo dataset.

In [0]:
# Caminho para salvar o DataFrame refinado (camada prata)
file_path_silver = "/mnt/dados/camada_prata_instalacoes"

# Removendo o diretório existente
dbutils.fs.rm(file_path_silver, recurse=True)

# Salvando o DataFrame refinado no formato Delta Lake (camada prata)
df.write.format("delta").mode("overwrite").save(file_path_silver)
